Dans ce code : Les données AWAY et HOME sont combinées. Des nouvelles variables sont créées
(Variables trop corrélées et gestion des NA grâce a al ligue quelquesoit le code)


In [ ]:

import pandas as pd
import numpy as np
import seaborn as sns
#!pip install --upgrade --force-reinstall matplotlib
import matplotlib.pyplot as plt
import warnings
import pandas as pd
import numpy as np
#!pip install tqdm 
#!pip install xgboost
from tqdm import tqdm
#!pip install scikit-learn
#!pip install xgboost
from sklearn import model_selection
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import cross_val_score
import xgboost as xgb
import lightgbm as lgb
import warnings
import matplotlib.pyplot as plt
#!pip install pycaret

# **Chargement des csv**

In [2]:
train_home_team = pd.read_csv('C:/Users/agraf/Desktop/uni/master/volume 2 foot (1)/volume 2 foot/train_home_team_statistics_df.csv', index_col=0)
train_away_team = pd.read_csv('C:/Users/agraf/Desktop/uni/master/volume 2 foot (1)/volume 2 foot/train_away_team_statistics_df.csv', index_col=0)

train_home_player = pd.read_csv('C:/Users/agraf/Desktop/uni/master/volume 2 foot (1)/volume 2 foot/train_home_player_statistics_df.csv', index_col=0)
train_away_player = pd.read_csv('C:/Users/agraf/Desktop/uni/master/volume 2 foot (1)/volume 2 foot/train_away_player_statistics_df.csv', index_col=0)

train_scores = pd.read_csv('C:/Users/agraf/Desktop/uni/master/volume 2 foot (1)/volume 2 foot/Y_train.csv', index_col=0)

test_home_team = pd.read_csv('C:/Users/agraf/Desktop/uni/master/volume 2 foot (1)/volume 2 foot/test_home_team_statistics_df.csv', index_col=0)
test_away_team = pd.read_csv('C:/Users/agraf/Desktop/uni/master/volume 2 foot (1)/volume 2 foot/test_away_team_statistics_df.csv', index_col=0)

Gerer les NA avec la moyenne de chaque ligue et de chaque DB pour pouvoir les fusionner, ainsi que enlever league (une fois l'avoir utilisée) ainsi que TEAM_NAME car inutile dans le challenge comme expliqué, + les inf.


In [3]:
# Drop column: 'TEAM_NAME'
train_home_team = train_home_team.drop(columns=['TEAM_NAME'])
# Pour train_home_team
train_home_team = train_home_team.replace({np.inf: np.nan, -np.inf: np.nan})

# Pour train_away_team
train_away_team = train_away_team.replace({np.inf: np.nan, -np.inf: np.nan})

# Pour test_home_team
test_home_team = test_home_team.replace({np.inf: np.nan, -np.inf: np.nan})

# Pour test_away_team
test_away_team = test_away_team.replace({np.inf: np.nan, -np.inf: np.nan})



In [4]:
#Voir les différentes league :
unique_leagues = train_home_team['LEAGUE'].unique()
print(unique_leagues)

#Calculer la moyenne de chaque colonne par ligue
means_per_league_home = train_home_team.groupby('LEAGUE').transform('mean')

# Remplacer les valeurs NA par les moyennes de la ligue correspondante
train_home_team = train_home_team.fillna(means_per_league_home)

print(train_home_team)

#On enleve "TEAM_INJURIES_5_last_match_std" car elle contient quand meme des NA, ainsi que league car la variable ne nous sert plus
train_home_team = train_home_team.drop(columns=['TEAM_INJURIES_5_last_match_std'])
train_home_team = train_home_team.drop(columns=['LEAGUE'])


['Ligue 1' 'Ligue 2' 'Serie A' 'League One' 'Premier League'
 'Liga Portugal' 'La Liga' 'Superliga' 'Bundesliga' 'Pro League'
 'J-League' 'League Two' 'Eredivisie']
               LEAGUE  TEAM_SHOTS_TOTAL_season_sum  \
ID                                                   
0             Ligue 1                          3.0   
1             Ligue 2                          6.0   
2             Serie A                          4.0   
3          League One                          7.0   
4      Premier League                          3.0   
...               ...                          ...   
12298      League One                          4.0   
12299   Liga Portugal                          4.0   
12300      Bundesliga                          4.0   
12301      League One                          2.0   
12302         La Liga                          2.0   

       TEAM_SHOTS_INSIDEBOX_season_sum  TEAM_SHOTS_OFF_TARGET_season_sum  \
ID                                                      

In [5]:
# Pour train_away_team
# Drop column: 'TEAM_NAME'
train_away_team = train_away_team.drop(columns=['TEAM_NAME'])

# Calculer la moyenne de chaque colonne par ligue
means_per_league_away = train_away_team.groupby('LEAGUE').transform('mean')

# Remplacer les valeurs NA par les moyennes de la ligue correspondante
train_away_team = train_away_team.fillna(means_per_league_away)

# On enlève "TEAM_INJURIES_5_last_match_std" et 'LEAGUE'
train_away_team = train_away_team.drop(columns=['TEAM_INJURIES_5_last_match_std'])
train_away_team = train_away_team.drop(columns=['LEAGUE'])

# Pour test_home_team

# Remplacer les valeurs NA par les moyennes correspondantes à chaque colonne des équipes home
test_home_team = test_home_team.fillna(means_per_league_home.mean())

# On enlève "TEAM_INJURIES_5_last_match_std"
test_home_team = test_home_team.drop(columns=['TEAM_INJURIES_5_last_match_std'])

# Pour test_away_team

# Remplacer les valeurs NA par les moyennes correspondantes à chaque colonne des équipes away
test_away_team = test_away_team.fillna(means_per_league_away.mean())

# On enlève "TEAM_INJURIES_5_last_match_std"
test_away_team = test_away_team.drop(columns=['TEAM_INJURIES_5_last_match_std'])

Je renomme les colonnes pour pouvoir différencier le home du away

In [6]:
train_home_team.columns = 'HOME_' + train_home_team.columns
train_away_team.columns = 'AWAY_' + train_away_team.columns

test_home_team.columns = 'HOME_' + test_home_team.columns
test_away_team.columns = 'AWAY_' + test_away_team.columns

Puis je réunis les deux train et test ensemble, ainsi que mes Y

In [7]:

train_data =  pd.concat([train_home_team,train_away_team],join='inner',axis=1)
train_scores = train_scores.loc[train_data.index]
train_data = pd.merge(train_data, train_scores, on='ID')

train_data['target_variable'] = train_data[['HOME_WINS', 'DRAW', 'AWAY_WINS']].idxmax(axis=1)
target_mapping = {'HOME_WINS': 0, 'DRAW': 1, 'AWAY_WINS': 2}
train_data['target_variable'] = train_data['target_variable'].map(target_mapping)
train_data.drop(columns=['HOME_WINS', 'DRAW', 'AWAY_WINS'], inplace=True)

test_data =  pd.concat([test_home_team,test_away_team],join='inner',axis=1)



In [8]:
pd.set_option('display.max_columns', None)
train_data.head()

,HOME_TEAM_SHOTS_TOTAL_season_sum,HOME_TEAM_SHOTS_INSIDEBOX_season_sum,HOME_TEAM_SHOTS_OFF_TARGET_season_sum,HOME_TEAM_SHOTS_ON_TARGET_season_sum,HOME_TEAM_SHOTS_OUTSIDEBOX_season_sum,HOME_TEAM_PASSES_season_sum,HOME_TEAM_SUCCESSFUL_PASSES_season_sum,HOME_TEAM_SAVES_season_sum,HOME_TEAM_CORNERS_season_sum,HOME_TEAM_FOULS_season_sum,HOME_TEAM_YELLOWCARDS_season_sum,HOME_TEAM_REDCARDS_season_sum,HOME_TEAM_OFFSIDES_season_sum,HOME_TEAM_ATTACKS_season_sum,HOME_TEAM_PENALTIES_season_sum,HOME_TEAM_SUBSTITUTIONS_season_sum,HOME_TEAM_BALL_SAFE_season_sum,HOME_TEAM_DANGEROUS_ATTACKS_season_sum,HOME_TEAM_INJURIES_season_sum,HOME_TEAM_GOALS_season_sum,HOME_TEAM_GAME_WON_season_sum,HOME_TEAM_GAME_DRAW_season_sum,HOME_TEAM_GAME_LOST_season_sum,HOME_TEAM_SHOTS_TOTAL_season_average,HOME_TEAM_SHOTS_INSIDEBOX_season_average,HOME_TEAM_SHOTS_OFF_TARGET_season_average,HOME_TEAM_SHOTS_ON_TARGET_season_average,HOME_TEAM_SHOTS_OUTSIDEBOX_season_average,HOME_TEAM_PASSES_season_average,HOME_TEAM_SUCCESSFUL_PASSES_season_average,HOME_TEAM_SUCCESSFUL_PASSES_PERCENTAGE_season_average,HOME_TEAM_SAVES_season_average,HOME_TEAM_CORNERS_season_average,HOME_TEAM_BALL_POSSESSION_season_average,HOME_TEAM_FOULS_season_average,HOME_TEAM_YELLOWCARDS_season_average,HOME_TEAM_REDCARDS_season_average,HOME_TEAM_OFFSIDES_season_average,HOME_TEAM_ATTACKS_season_average,HOME_TEAM_PENALTIES_season_average,HOME_TEAM_SUBSTITUTIONS_season_average,HOME_TEAM_BALL_SAFE_season_average,HOME_TEAM_DANGEROUS_ATTACKS_season_average,HOME_TEAM_INJURIES_season_average,HOME_TEAM_GOALS_season_average,HOME_TEAM_GAME_WON_season_average,HOME_TEAM_GAME_DRAW_season_average,HOME_TEAM_GAME_LOST_season_average,HOME_TEAM_SHOTS_TOTAL_season_std,HOME_TEAM_SHOTS_INSIDEBOX_season_std,HOME_TEAM_SHOTS_OFF_TARGET_season_std,HOME_TEAM_SHOTS_ON_TARGET_season_std,HOME_TEAM_SHOTS_OUTSIDEBOX_season_std,HOME_TEAM_PASSES_season_std,HOME_TEAM_SUCCESSFUL_PASSES_season_std,HOME_TEAM_SUCCESSFUL_PASSES_PERCENTAGE_season_std,HOME_TEAM_SAVES_season_std,HOME_TEAM_CORNERS_season_std,HOME_TEAM_BALL_POSSESSION_season_std,HOME_TEAM_FOULS_season_std,HOME_TEAM_YELLOWCARDS_season_std,HOME_TEAM_REDCARDS_season_std,HOME_TEAM_OFFSIDES_season_std,HOME_TEAM_ATTACKS_season_std,HOME_TEAM_PENALTIES_season_std,HOME_TEAM_SUBSTITUTIONS_season_std,HOME_TEAM_BALL_SAFE_season_std,HOME_TEAM_DANGEROUS_ATTACKS_season_std,HOME_TEAM_INJURIES_season_std,HOME_TEAM_GOALS_season_std,HOME_TEAM_SHOTS_TOTAL_5_last_match_sum,HOME_TEAM_SHOTS_INSIDEBOX_5_last_match_sum,HOME_TEAM_SHOTS_OFF_TARGET_5_last_match_sum,HOME_TEAM_SHOTS_ON_TARGET_5_last_match_sum,HOME_TEAM_SHOTS_OUTSIDEBOX_5_last_match_sum,HOME_TEAM_PASSES_5_last_match_sum,HOME_TEAM_SUCCESSFUL_PASSES_5_last_match_sum,HOME_TEAM_SAVES_5_last_match_sum,HOME_TEAM_CORNERS_5_last_match_sum,HOME_TEAM_FOULS_5_last_match_sum,HOME_TEAM_YELLOWCARDS_5_last_match_sum,HOME_TEAM_REDCARDS_5_last_match_sum,HOME_TEAM_OFFSIDES_5_last_match_sum,HOME_TEAM_ATTACKS_5_last_match_sum,HOME_TEAM_PENALTIES_5_last_match_sum,HOME_TEAM_SUBSTITUTIONS_5_last_match_sum,HOME_TEAM_BALL_SAFE_5_last_match_sum,HOME_TEAM_DANGEROUS_ATTACKS_5_last_match_sum,HOME_TEAM_INJURIES_5_last_match_sum,HOME_TEAM_GOALS_5_last_match_sum,HOME_TEAM_GAME_WON_5_last_match_sum,HOME_TEAM_GAME_DRAW_5_last_match_sum,HOME_TEAM_GAME_LOST_5_last_match_sum,HOME_TEAM_SHOTS_TOTAL_5_last_match_average,HOME_TEAM_SHOTS_INSIDEBOX_5_last_match_average,HOME_TEAM_SHOTS_OFF_TARGET_5_last_match_average,HOME_TEAM_SHOTS_ON_TARGET_5_last_match_average,HOME_TEAM_SHOTS_OUTSIDEBOX_5_last_match_average,HOME_TEAM_PASSES_5_last_match_average,HOME_TEAM_SUCCESSFUL_PASSES_5_last_match_average,HOME_TEAM_SUCCESSFUL_PASSES_PERCENTAGE_5_last_match_average,HOME_TEAM_SAVES_5_last_match_average,HOME_TEAM_CORNERS_5_last_match_average,HOME_TEAM_BALL_POSSESSION_5_last_match_average,HOME_TEAM_FOULS_5_last_match_average,HOME_TEAM_YELLOWCARDS_5_last_match_average,HOME_TEAM_REDCARDS_5_last_match_average,HOME_TEAM_OFFSIDES_5_last_match_average,HOME_TEAM_ATTACKS_5_last_match_average,HOME_TEAM_PENALTI

# Creation de nouvelles variables :


J'ajoute des variables que je calcule grace aux données dans Team

In [9]:
# Création d'une copie de sauvegarde de la base de données originale
train_data_backup = train_data.copy()

# Création de la colonne 'HOME_TEAM_SHOTS_TOTAL_season_sum' (somme des tirs de l'équipe à domicile)
train_data['HOME_TEAM_SHOTS_TOTAL_season_sum'] = train_data['HOME_TEAM_SHOTS_INSIDEBOX_season_sum'] + train_data['HOME_TEAM_SHOTS_OUTSIDEBOX_season_sum']

# Création de la colonne 'AWAY_TEAM_SHOTS_TOTAL_season_sum' (somme des tirs de l'équipe à l'extérieur)
train_data['AWAY_TEAM_SHOTS_TOTAL_season_sum'] = train_data['AWAY_TEAM_SHOTS_INSIDEBOX_season_sum'] + train_data['AWAY_TEAM_SHOTS_OUTSIDEBOX_season_sum']

# 1. Différentiel de tirs (cumulé, moyenne, sur les 5 derniers matchs)
train_data['DIFF_SHOTS_ON_TARGET'] = train_data['HOME_TEAM_SHOTS_ON_TARGET_season_sum'] - train_data['AWAY_TEAM_SHOTS_ON_TARGET_season_sum']
train_data['DIFF_SHOTS_ON_TARGET_5_LAST'] = train_data['HOME_TEAM_SHOTS_ON_TARGET_5_last_match_sum'] - train_data['AWAY_TEAM_SHOTS_ON_TARGET_5_last_match_sum']

# 2. Possession de balle relative (cumulée, moyenne, sur les 5 derniers matchs)
train_data['DIFF_BALL_POSSESSION'] = train_data['HOME_TEAM_BALL_POSSESSION_season_average'] - train_data['AWAY_TEAM_BALL_POSSESSION_season_average']
train_data['DIFF_BALL_POSSESSION_5_LAST'] = train_data['HOME_TEAM_BALL_POSSESSION_5_last_match_average'] - train_data['AWAY_TEAM_BALL_POSSESSION_5_last_match_average']

# 8. Différentiel de tirs dangereux (cumulé et sur les 5 derniers matchs)
train_data['DIFF_DANGEROUS_ATTACKS'] = train_data['HOME_TEAM_DANGEROUS_ATTACKS_season_sum'] - train_data['AWAY_TEAM_DANGEROUS_ATTACKS_season_sum']
train_data['DIFF_DANGEROUS_ATTACKS_5_LAST'] = train_data['HOME_TEAM_DANGEROUS_ATTACKS_5_last_match_sum'] - train_data['AWAY_TEAM_DANGEROUS_ATTACKS_5_last_match_sum']

# 10. Ratio tirs sur cible/tirs totaux (cumulé, moyenne, sur les 5 derniers matchs)
train_data['HOME_RATIO_SHOTS_ON_TARGET'] = train_data['HOME_TEAM_SHOTS_ON_TARGET_season_sum'] / train_data['HOME_TEAM_SHOTS_TOTAL_season_sum']
train_data['AWAY_RATIO_SHOTS_ON_TARGET'] = train_data['AWAY_TEAM_SHOTS_ON_TARGET_season_sum'] / train_data['AWAY_TEAM_SHOTS_TOTAL_season_sum']
train_data['DIFF_RATIO_SHOTS_ON_TARGET'] = train_data['HOME_RATIO_SHOTS_ON_TARGET'] - train_data['AWAY_RATIO_SHOTS_ON_TARGET']
train_data['HOME_RATIO_SHOTS_ON_TARGET_5_LAST'] = train_data['HOME_TEAM_SHOTS_ON_TARGET_5_last_match_sum'] / (train_data['HOME_TEAM_SHOTS_INSIDEBOX_5_last_match_sum'] + train_data['HOME_TEAM_SHOTS_OUTSIDEBOX_5_last_match_sum'])
train_data['AWAY_RATIO_SHOTS_ON_TARGET_5_LAST'] = train_data['AWAY_TEAM_SHOTS_ON_TARGET_5_last_match_sum'] / (train_data['AWAY_TEAM_SHOTS_INSIDEBOX_5_last_match_sum'] + train_data['AWAY_TEAM_SHOTS_OUTSIDEBOX_5_last_match_sum'])
train_data['DIFF_RATIO_SHOTS_ON_TARGET_5_LAST'] = train_data['HOME_RATIO_SHOTS_ON_TARGET_5_LAST'] - train_data['AWAY_RATIO_SHOTS_ON_TARGET_5_LAST']

# 13. Différence de réussite offensive (but par tir)
train_data['DIFF_GOALS_PER_SHOT'] = (train_data['HOME_TEAM_GOALS_season_sum'] / train_data['HOME_TEAM_SHOTS_TOTAL_season_sum']) - (train_data['AWAY_TEAM_GOALS_season_sum'] / train_data['AWAY_TEAM_SHOTS_TOTAL_season_sum'])

# 24. Ratio "Attaques dangereuses" / "Attaques totales"
train_data['HOME_RATIO_DANGEROUS_ATTACKS'] = train_data['HOME_TEAM_DANGEROUS_ATTACKS_season_sum'] / train_data['HOME_TEAM_ATTACKS_season_sum']
train_data['AWAY_RATIO_DANGEROUS_ATTACKS'] = train_data['AWAY_TEAM_DANGEROUS_ATTACKS_season_sum'] / train_data['AWAY_TEAM_ATTACKS_season_sum']
train_data['DIFF_RATIO_DANGEROUS_ATTACKS'] = train_data['HOME_RATIO_DANGEROUS_ATTACKS'] - train_data['AWAY_RATIO_DANGEROUS_ATTACKS']

# Momentum des équipes
train_data['MOMENTUM_TEAM_HOME'] = train_data['HOME_TEAM_GAME_WON_5_last_match_average'] - train_data['HOME_TEAM_GAME_WON_season_average']
train_data['MOMENTUM_TEAM_AWAY'] = train_data['AWAY_TEAM_GAME_WON_5_last_match_average'] - train_data['AWAY_TEAM_GAME_WON_season_average']

# Sauvegarde du DataFrame modifié
#train_data.to_csv('train_data_modified.csv', index=False)



Certaine de ces création donnent des NA, je les enleve donc car cela n'est pas une variable utilisable

In [10]:
#Je fais pareil pour ma base test pour que mes modèles fonctionnent
test_data_backup = test_data.copy()

# Création de la colonne 'HOME_TEAM_SHOTS_TOTAL_season_sum' (somme des tirs de l'équipe à domicile)
test_data['HOME_TEAM_SHOTS_TOTAL_season_sum'] = test_data['HOME_TEAM_SHOTS_INSIDEBOX_season_sum'] + test_data['HOME_TEAM_SHOTS_OUTSIDEBOX_season_sum']

# Création de la colonne 'AWAY_TEAM_SHOTS_TOTAL_season_sum' (somme des tirs de l'équipe à l'extérieur)
test_data['AWAY_TEAM_SHOTS_TOTAL_season_sum'] = test_data['AWAY_TEAM_SHOTS_INSIDEBOX_season_sum'] + test_data['AWAY_TEAM_SHOTS_OUTSIDEBOX_season_sum']

# 1. Différentiel de tirs (cumulé, moyenne, sur les 5 derniers matchs)
test_data['DIFF_SHOTS_ON_TARGET'] = test_data['HOME_TEAM_SHOTS_ON_TARGET_season_sum'] - test_data['AWAY_TEAM_SHOTS_ON_TARGET_season_sum']
test_data['DIFF_SHOTS_ON_TARGET_5_LAST'] = test_data['HOME_TEAM_SHOTS_ON_TARGET_5_last_match_sum'] - test_data['AWAY_TEAM_SHOTS_ON_TARGET_5_last_match_sum']

# 2. Possession de balle relative (cumulée, moyenne, sur les 5 derniers matchs)
test_data['DIFF_BALL_POSSESSION'] = test_data['HOME_TEAM_BALL_POSSESSION_season_average'] - test_data['AWAY_TEAM_BALL_POSSESSION_season_average']
test_data['DIFF_BALL_POSSESSION_5_LAST'] = test_data['HOME_TEAM_BALL_POSSESSION_5_last_match_average'] - test_data['AWAY_TEAM_BALL_POSSESSION_5_last_match_average']

# 8. Différentiel de tirs dangereux (cumulé et sur les 5 derniers matchs)
test_data['DIFF_DANGEROUS_ATTACKS'] = test_data['HOME_TEAM_DANGEROUS_ATTACKS_season_sum'] - test_data['AWAY_TEAM_DANGEROUS_ATTACKS_season_sum']
test_data['DIFF_DANGEROUS_ATTACKS_5_LAST'] = test_data['HOME_TEAM_DANGEROUS_ATTACKS_5_last_match_sum'] - test_data['AWAY_TEAM_DANGEROUS_ATTACKS_5_last_match_sum']

# 10. Ratio tirs sur cible/tirs totaux (cumulé, moyenne, sur les 5 derniers matchs)
test_data['HOME_RATIO_SHOTS_ON_TARGET'] = test_data['HOME_TEAM_SHOTS_ON_TARGET_season_sum'] / test_data['HOME_TEAM_SHOTS_TOTAL_season_sum']
test_data['AWAY_RATIO_SHOTS_ON_TARGET'] = test_data['AWAY_TEAM_SHOTS_ON_TARGET_season_sum'] / test_data['AWAY_TEAM_SHOTS_TOTAL_season_sum']
test_data['DIFF_RATIO_SHOTS_ON_TARGET'] = test_data['HOME_RATIO_SHOTS_ON_TARGET'] - test_data['AWAY_RATIO_SHOTS_ON_TARGET']
test_data['HOME_RATIO_SHOTS_ON_TARGET_5_LAST'] = test_data['HOME_TEAM_SHOTS_ON_TARGET_5_last_match_sum'] / (test_data['HOME_TEAM_SHOTS_INSIDEBOX_5_last_match_sum'] + test_data['HOME_TEAM_SHOTS_OUTSIDEBOX_5_last_match_sum'])
test_data['AWAY_RATIO_SHOTS_ON_TARGET_5_LAST'] = test_data['AWAY_TEAM_SHOTS_ON_TARGET_5_last_match_sum'] / (test_data['AWAY_TEAM_SHOTS_INSIDEBOX_5_last_match_sum'] + test_data['AWAY_TEAM_SHOTS_OUTSIDEBOX_5_last_match_sum'])
test_data['DIFF_RATIO_SHOTS_ON_TARGET_5_LAST'] = test_data['HOME_RATIO_SHOTS_ON_TARGET_5_LAST'] - test_data['AWAY_RATIO_SHOTS_ON_TARGET_5_LAST']

# 13. Différence de réussite offensive (but par tir)
test_data['DIFF_GOALS_PER_SHOT'] = (test_data['HOME_TEAM_GOALS_season_sum'] / test_data['HOME_TEAM_SHOTS_TOTAL_season_sum']) - (test_data['AWAY_TEAM_GOALS_season_sum'] / test_data['AWAY_TEAM_SHOTS_TOTAL_season_sum'])

# 24. Ratio "Attaques dangereuses" / "Attaques totales"
test_data['HOME_RATIO_DANGEROUS_ATTACKS'] = test_data['HOME_TEAM_DANGEROUS_ATTACKS_season_sum'] / test_data['HOME_TEAM_ATTACKS_season_sum']
test_data['AWAY_RATIO_DANGEROUS_ATTACKS'] = test_data['AWAY_TEAM_DANGEROUS_ATTACKS_season_sum'] / test_data['AWAY_TEAM_ATTACKS_season_sum']
test_data['DIFF_RATIO_DANGEROUS_ATTACKS'] = test_data['HOME_RATIO_DANGEROUS_ATTACKS'] - test_data['AWAY_RATIO_DANGEROUS_ATTACKS']

# Momentum des équipes
test_data['MOMENTUM_TEAM_HOME'] = test_data['HOME_TEAM_GAME_WON_5_last_match_average'] - test_data['HOME_TEAM_GAME_WON_season_average']
test_data['MOMENTUM_TEAM_AWAY'] = test_data['AWAY_TEAM_GAME_WON_5_last_match_average'] - test_data['AWAY_TEAM_GAME_WON_season_average']

C:\Users\agraf\AppData\Local\Temp\ipykernel_12188\1553872936.py:11: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  test_data['DIFF_SHOTS_ON_TARGET'] = test_data['HOME_TEAM_SHOTS_ON_TARGET_season_sum'] - test_data['AWAY_TEAM_SHOTS_ON_TARGET_season_sum']
C:\Users\agraf\AppData\Local\Temp\ipykernel_12188\1553872936.py:12: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  test_data['DIFF_SHOTS_ON_TARGET_5_LAST'] = test_data['HOME_TEAM_SHOTS_ON_TARGET_5_last_match_sum'] - test_data['AWAY_TEAM_SHOTS_ON_TARGET_5_last_match_sum']
C:\Users\ag

In [11]:
# Supprimer les colonnes contenant des NA dans train_data
train_data = train_data.dropna(axis=1)

# Supprimer les colonnes contenant des NA dans test_data
test_data = test_data.dropna(axis=1)

Coefficient de correlation pour enlever certaines variables trop corrélées entre elles

In [12]:
# Définir la variable cible (Y)
target_variable = 'target_variable'

# Calculer la matrice de corrélation sans la variable cible
corr_matrix = train_data.drop(columns=[target_variable]).corr().abs()

# Supprimer les variables ayant une corrélation supérieure à 0.8
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
to_drop = [column for column in upper.columns if any(upper[column] > 0.8)]
print(f'Nombre de variables supprimées : {len(to_drop)}')
print(f'Variables supprimées : {to_drop}')

# Supprimer les variables corrélées dans le DataFrame d'origine
train_data = train_data.drop(columns=to_drop)
test_data = test_data.drop(columns=to_drop, errors='ignore')

Nombre de variables supprimées : 121
Variables supprimées : ['HOME_TEAM_SHOTS_INSIDEBOX_season_sum', 'HOME_TEAM_SHOTS_OUTSIDEBOX_season_sum', 'HOME_TEAM_SUCCESSFUL_PASSES_season_sum', 'HOME_TEAM_SHOTS_TOTAL_season_average', 'HOME_TEAM_SHOTS_INSIDEBOX_season_average', 'HOME_TEAM_SHOTS_OFF_TARGET_season_average', 'HOME_TEAM_SHOTS_ON_TARGET_season_average', 'HOME_TEAM_SHOTS_OUTSIDEBOX_season_average', 'HOME_TEAM_PASSES_season_average', 'HOME_TEAM_SUCCESSFUL_PASSES_season_average', 'HOME_TEAM_SUCCESSFUL_PASSES_PERCENTAGE_season_average', 'HOME_TEAM_SAVES_season_average', 'HOME_TEAM_CORNERS_season_average', 'HOME_TEAM_BALL_POSSESSION_season_average', 'HOME_TEAM_FOULS_season_average', 'HOME_TEAM_YELLOWCARDS_season_average', 'HOME_TEAM_REDCARDS_season_average', 'HOME_TEAM_OFFSIDES_season_average', 'HOME_TEAM_ATTACKS_season_average', 'HOME_TEAM_PENALTIES_season_average', 'HOME_TEAM_SUBSTITUTIONS_season_average', 'HOME_TEAM_DANGEROUS_ATTACKS_season_average', 'HOME_TEAM_INJURIES_season_average',

Creation des modeles : (le choix des parametres est fait pour allier une précision maximale tout en évitant le sur-apprentissage)
Les modeles ci dessous ne sont pas dans l'ordre de ceux réalisés pour le challenge. Le premier fut un random forest

XGBOOST

In [ ]:
#XGBOOST
# Définir les données d'entraînement
target_variable = 'target_variable'  
X_train = train_data.drop(columns=[target_variable])
y_train = train_data[target_variable]

# Définir les hyperparamètres à ajuster
param_distributions = {
    'n_estimators': [50, 100, 200, 300],  # Nombre d'arbres dans le modèle
    'max_depth': [3, 5, 7, 10],  # Profondeur maximale des arbres
    'learning_rate': [0.01, 0.05, 0.1, 0.2],  # Taux d'apprentissage
    'subsample': [0.6, 0.8, 1.0],  # Fraction d'échantillons utilisés pour entraîner chaque arbre
    'colsample_bytree': [0.6, 0.8, 1.0],  # Fraction des caractéristiques utilisées par arbre
    'gamma': [0, 0.1, 0.3, 0.5],  # Paramètre de régularisation
    'min_child_weight': [1, 3, 5]  # Poids minimum de l'enfant
}

# Créer une instance du modèle XGBoost
xgb_model = xgb.XGBClassifier(objective='multi:softmax', num_class=3, random_state=64)

# Utiliser RandomizedSearchCV pour optimiser les hyperparamètres
random_search = RandomizedSearchCV(
    estimator=xgb_model,
    param_distributions=param_distributions,
    n_iter=20,  # Nombre d'itérations (pas trop haute pour le temps de calcul)
    cv=3,  # Validation croisée à 3 folds
    verbose=2,
    random_state=64,
    n_jobs=-1  # Utiliser tous les processeurs disponibles
)

# Entraîner la recherche
random_search.fit(X_train, y_train)

# Afficher les meilleurs hyperparamètres trouvés
print("Meilleurs hyperparamètres trouvés :")
print(random_search.best_params_)

# Utiliser le meilleur modèle trouvé pour faire des prédictions
best_xgb = random_search.best_estimator_

# Prédire la variable cible pour la base test_data (les variables explicatives)
y_pred_test = best_xgb.predict(test_data)

# Créer une matrice de zéros pour les prédictions one-hot
y_pred_one_hot = np.zeros((len(y_pred_test), 3), dtype=int)

# Remplir la matrice one-hot en fonction des prédictions
for i, pred in enumerate(y_pred_test):
    y_pred_one_hot[i, pred] = 1

# Créer un DataFrame pour les résultats de prédiction
predictions_one_hot_df = pd.DataFrame(y_pred_one_hot, columns=['HOME_WINS', 'DRAW', 'AWAY_WINS'])

# Ajouter une colonne ID (utiliser l'index de test_data comme identifiant)
predictions_one_hot_df.insert(0, 'ID', test_data.index)

# Exporter les résultats dans un fichier CSV
predictions_one_hot_df.to_csv('predictions_xgboost_best_one_hot.csv', index=False)

print("Prédictions one-hot enregistrées dans le fichier 'predictions_xgboost_best_one_hot.csv'")


Fitting 3 folds for each of 20 candidates, totalling 60 fits
Meilleurs hyperparamètres trouvés :
{'subsample': 0.8, 'n_estimators': 50, 'min_child_weight': 5, 'max_depth': 7, 'learning_rate': 0.05, 'gamma': 0.3, 'colsample_bytree': 0.8}
Prédictions one-hot enregistrées dans le fichier 'predictions_xgboost_best_one_hot.csv'


Random Forest

In [ ]:
# Définir des hyperparametres pour les tester

seed = 64

target_variable = 'target_variable' 
X_train = train_data.drop(columns=[target_variable])
y_train = train_data[target_variable]

rf = RandomForestClassifier(random_state=seed)

param_distributions = {
    'n_estimators': [50, 100, 200],  
    'max_features': ['sqrt', 'log2'],  
    'max_depth': [5, 8, 10],  
    'min_samples_split': [10, 15, 20],  
    'min_samples_leaf': [4, 6, 8],  
    'bootstrap': [True]
}

# Utiliser RandomizedSearchCV pour trouver les meilleurs hyperparamètres
rf_random = RandomizedSearchCV(estimator=rf, param_distributions=param_distributions,
                               n_iter=20, cv=3, verbose=2, random_state=42, n_jobs=-1)

# Entraîner à nouveau
rf_random.fit(X_train, y_train)

# Afficher les meilleurs hyperparamètres
print("Meilleurs hyperparamètres trouvés :")
print(rf_random.best_params_)

# Prédire sur les données d'entraînement avec les meilleurs hyperparamètres trouvés
best_rf = rf_random.best_estimator_
y_pred_train = best_rf.predict(X_train)

# Calculer et afficher la précision sur les données d'entraînement
accuracy_train = accuracy_score(y_train, y_pred_train)
print(f'Précision sur les données d\'entraînement après ajustement : {accuracy_train:.4f}')


In [ ]:
#autres test pour voir

seed = 64

target_variable = 'target_variable' 
X_train = train_data.drop(columns=[target_variable])
y_train = train_data[target_variable]

rf = RandomForestClassifier(random_state=seed)

param_distributions = {
    'n_estimators': [50, 100, 150, 125, 75, 200], 
    'max_features': ['sqrt', 'log2', 0.4, 0.3, 0.5, 0.2],
    'max_depth': [7, 8, 9, 10, 11, 12, 15, 20],
    'min_samples_split': [8, 9, 10, 12, 14, 11, 13], 
    'min_samples_leaf': [4, 5, 6, 7, 8, 9, 10], 
    'bootstrap': [True]
}

# Utiliser RandomizedSearchCV pour trouver les meilleurs hyperparamètres
rf_random = RandomizedSearchCV(estimator=rf, param_distributions=param_distributions,
                               n_iter=25, cv=3, verbose=2, random_state=42, n_jobs=-1)

# Entraîner à nouveau
rf_random.fit(X_train, y_train)

# Afficher les meilleurs hyperparamètres
print("Meilleurs hyperparamètres trouvés :")
print(rf_random.best_params_)

# Prédire sur les données d'entraînement avec les meilleurs hyperparamètres trouvés
best_rf2 = rf_random.best_estimator_
y_pred_train = best_rf2.predict(X_train)

# Calculer et afficher la précision sur les données d'entraînement
accuracy_train = accuracy_score(y_train, y_pred_train)
print(f'Précision sur les données d\'entraînement après ajustement : {accuracy_train:.4f}')


Validation croisée pour pouvoir comparer mes modéles entre eux, dans le but de faire la meilleur soumission au lieu de tous les essayer

In [ ]:
# Effectuer une validation croisée sur le modèle optimisé
cv_scores = cross_val_score(best_rf2, X_train, y_train, cv=5, scoring='accuracy')

# Calculer la précision moyenne et l'écart-type
cv_mean = np.mean(cv_scores)
cv_std = np.std(cv_scores)

print(f'Précision moyenne avec validation croisée à 5 folds : {cv_mean:.4f}')
print(f'Écart-type des précisions : {cv_std:.4f}')

Importance des variables dans le modèle : Le but est in fine d'enlever les variables ne contribuant pas/peu pour pouvoir améliorer le modèle, ou de combiner certaine des variables si elles me semblent significative

In [ ]:
# Accéder à l'importance des variables dans le modèle entraîné
feature_importances = best_rf2.feature_importances_

# Créer un DataFrame pour associer les importances aux noms des caractéristiques
features = X_train.columns  # Les noms des variables explicatives
importance_df = pd.DataFrame({'Feature': features, 'Importance': feature_importances})

# Trier les variables par ordre décroissant d'importance
importance_df = importance_df.sort_values(by='Importance', ascending=False)

# Afficher les variables les plus importantes
print(importance_df)

# Visualiser l'importance des variables avec un graphique en barres
plt.figure(figsize=(10, 8))
plt.barh(importance_df['Feature'][:15], importance_df['Importance'][:15], color='skyblue')
plt.xlabel('Importance')
plt.ylabel('Feature')
plt.title('Top 15 Variables les Plus Importantes')
plt.gca().invert_yaxis()  
plt.show()

Pour exporter les variables (ne pas prendre en compte)

In [19]:
# y_pred_test = best_rf2.predict(test_data)

# # Créer un DataFrame pour les résultats de prédiction
# predictions = pd.DataFrame({
#     'ID': test_data.index,  
#     'target_variable': y_pred_test
# })

# # Exporter les résultats dans un fichier CSV
# predictions.to_csv('predictions.csv', index=False)

# print("Prédictions enregistrées dans le fichier 'predictions.csv'")

In [20]:
# # Charger les prédictions depuis le fichier CSV 'y_pred_test'
# y_pred_df = pd.read_csv('predictions.csv')

# # Assumer que le fichier CSV contient une colonne 'target_variable' qui a les valeurs prédictes (0, 1, 2)
# y_pred = y_pred_df['target_variable']

# # Créer une matrice de zéros pour les prédictions one-hot
# # Chaque ligne représente un match, chaque colonne représente une classe : [HOME_WINS, DRAW, AWAY_WINS]
# y_pred_one_hot = np.zeros((len(y_pred), 3), dtype=int)

# # Remplir la matrice one-hot en fonction des prédictions
# for i, pred in enumerate(y_pred):
#     y_pred_one_hot[i, pred] = 1

# # Créer un DataFrame pour les résultats de prédiction
# predictions_one_hot_df = pd.DataFrame(y_pred_one_hot, columns=['HOME_WINS', 'DRAW', 'AWAY_WINS'])

# # Ajouter une colonne ID (si nécessaire, utiliser l'index du fichier initial)
# if 'ID' in y_pred_df.columns:
#     predictions_one_hot_df.insert(0, 'ID', y_pred_df['ID'])
# else:
#     predictions_one_hot_df.insert(0, 'ID', y_pred_df.index)

# # Exporter les résultats dans un nouveau fichier CSV
# predictions_one_hot_df.to_csv('predictions_one_hot.csv', index=False)

# print("Prédictions one-hot enregistrées dans le fichier 'predictions_one_hot.csv'")

In [21]:
# print("Nombre de prédictions : ", len(y_pred_test))
# print("Valeurs des prédictions : ", np.unique(y_pred_test, return_counts=True))

In [ ]:
#LightGBM

# Définir une valeur de seed pour la reproductibilité
seed = 64

# Charger et entraîner le modèle LightGBM
model = lgb.LGBMClassifier(objective='multiclass', num_class=3, random_state=64)
model.fit(X_train, y_train)  # Entraîner le modèle sur les données d'entraînement

# Prédire la variable cible pour la base test_data (les variables explicatives)
y_pred_test = model.predict(test_data)

# Créer une matrice de zéros pour les prédictions one-hot
# Chaque ligne représente un match, chaque colonne représente une classe : [HOME_WINS, DRAW, AWAY_WINS]
y_pred_one_hot = np.zeros((len(y_pred_test), 3), dtype=int)

# Remplir la matrice one-hot en fonction des prédictions
for i, pred in enumerate(y_pred_test):
    y_pred_one_hot[i, pred] = 1

# Créer un DataFrame pour les résultats de prédiction
predictions_one_hot_df = pd.DataFrame(y_pred_one_hot, columns=['HOME_WINS', 'DRAW', 'AWAY_WINS'])

# Ajouter une colonne ID (si nécessaire, utiliser l'index du fichier test_data comme identifiant)
predictions_one_hot_df.insert(0, 'ID', test_data.index)

# Exporter les résultats dans un nouveau fichier CSV avec des entiers
predictions_one_hot_df.to_csv('predictions_lightgbm_one_hot.csv', index=False)

print("Prédictions one-hot enregistrées dans le fichier 'predictions_lightgbm_one_hot.csv'")